In [ ]:
%pip install -r requirements.txt -q

In [3]:
# Импорт библиотек, классов и функций
import os
import json

import pandas as pd
import uuid
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split

In [4]:
# Загрузка и объвление констант
load_dotenv(".env")

# Таблица БД - хранилище признаков без очистки и преобразований
TBL = 'flats_fstore' 

# Креды для подключения к БД
PG_HOST = os.environ.get("PG_HOST")
PG_PORT = os.environ.get("PG_PORT")
PG_USER = os.environ.get("PG_USER")
PG_PASS = os.environ.get("PG_PASS")
PG_DB = os.environ.get("PG_DB")
CON_STR = f'postgresql://{PG_USER}:{PG_PASS}@{PG_HOST}:{PG_PORT}/{PG_DB}'

RANDOM_STATE = 42 # Ответ на главный вопрос жизни, вселенной и всего

# Размер тестовой выборки
TEST_SIZE = .25 

In [5]:
# Функция подключения к базе данных и получение датасета
def get_data(con_str=CON_STR, tbl=TBL):
    con = create_engine(con_str)
    result = pd.read_sql(f"select * from {tbl}", con)
    con.dispose()
    return result 

In [ ]:
data = get_data()
data.info()

In [ ]:
# Оценка значений для корзин гистограммы prometheus
data.query("price < 1.5e8")["price"].plot.hist(bins=100, figsize=(14, 5))


In [ ]:
# Разделение датасета
test = data.sample(500, random_state=RANDOM_STATE)
test.info()

In [9]:
# Сокращение датасета до n_count случайных записей с необходимыми признаками
n_count = 500
test.drop(columns="price", inplace=True)
# Добавление id для использования в качестве user_id
test["UUID"] = [uuid.uuid4() for _ in range(n_count)]
test.set_index("UUID", inplace=True)
# Сохранение в json файл в удобном формате (транспонированно)
test.T.to_json("../test_stress.json")

In [10]:
# Получение случайной записи для одиночного тестирования
test.sample(1, random_state=RANDOM_STATE).iloc[0].to_json("../test.json")

In [ ]:
# Проверка выгруженного 
with open("../test.json", "r") as fd:
    example = json.load(fd)
pd.DataFrame(example, index=[0])

In [ ]:
import joblib

with open("../../services/models/flats_price/outlier_processor.pkl", "rb") as fd:
    outl_proc = joblib.load(fd)

clean_data = outl_proc.transform(data)
clean_data

In [ ]:
clean_data = data.drop(columns=["id", "studio"])

clean_data["building_type_int"] = clean_data["building_type_int"].astype("str")
clean_data.rename(columns={"building_type_int": "building_type"}, inplace=True)
clean_data

In [ ]:
with open("../../services/models/flats_price/feat_processor.pkl", "rb") as fd:
    feat_proc = joblib.load(fd)
feat_proc.transform(clean_data.drop(columns=["price"]).sample(1, random_state=RANDOM_STATE))